In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

     |████████████████████████████████| 719 kB 29.3 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 365 kB 67.6 MB/s 
     |████████████████████████████████| 4.7 MB 64.6 MB/s 
     |████████████████████████████████| 212 kB 72.6 MB/s 
     |████████████████████████████████| 115 kB 68.0 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
     |████████████████████████████████| 6.6 MB 41.3 MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [3]:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files, # 이미지 파일로부터 아이템을 가져오는데
                   get_y=parent_label, # 부모는 파일의 폴더 명으로 레이블링 된다.
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64) # 배치 사이즈 64 

In [9]:
dls.c # 도대체 c는 뭐지?

10

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy) # 교차엔트로피, 평가지표 : 정확도
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.583717,3.737929,0.315534,02:59
1,1.254776,1.384987,0.591486,02:58
2,0.942539,0.836383,0.743092,02:57
3,0.725394,0.645134,0.794996,02:59
4,0.608696,0.550459,0.830471,02:57


In [ ]:
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3]) # 인덱스 1에 해당되는 채널 축은 제외

(TensorImage([0.4345, 0.4269, 0.3922], device='cuda:0'),
 TensorImage([0.2742, 0.2628, 0.2883], device='cuda:0'))

In [ ]:
def get_dls(bs, size):
  dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale = 0.75), # 사이즈를 인자로
                               Normalize.from_stats(*imagenet_stats)]) # 이미지넷의 자료를 가지고 정규화 평균 0 표준편차 1로 조정
  return dblock.dataloaders(path, bs=bs)

dls = get_dls(64, 224)

x, y = dls.one_batch()

x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([-0.0313,  0.0583,  0.1677], device='cuda:0'),
 TensorImage([1.2376, 1.2467, 1.3268], device='cuda:0'))

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.717172,3.103730,0.326736,03:03
1,1.279333,1.442635,0.512696,02:59
2,1.004035,0.855896,0.732263,02:58
3,0.755977,0.736605,0.761389,02:58
4,0.589611,0.567403,0.825243,02:57


In [ ]:
dls =get_dls(128, 128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.609498,2.208646,0.398058,02:26
1,1.234128,1.253801,0.638910,02:25
2,0.938919,0.942133,0.711725,02:24
3,0.741066,0.665690,0.788275,02:25


In [ ]:
# 속도가 느려서 더 이상 진행 못하겠음
# 향후 Fastai가 M1을 지원한다면 그때 다시 학습